In [1]:
# Dependencies
import requests
import os
import json
import zipfile

import pandas as pd
import numpy as np

# Extract

## Station Information

In [2]:
# # Citi Bike publishes real-time system data in GBFS format.
# gbfs_response = requests.get('http://gbfs.citibikenyc.com/gbfs/gbfs.json').json()
# station_information_json = requests.get(gbfs_response['data']['en']['feeds'][1]['url']).json()

# # Export the station information as a json file to avoid calling real-time data everytime this notebook is run.
# # Station info reflects the latest info upon calling the station API.  
# with open(os.path.join('data', 'station_info.json'), 'w') as f:
#     json.dump(station_information_json['data']['stations'], f)

## Trip Data

In [3]:
# Much like extraction of the station information, the following code needs to be executed only once to extract the trip data
# of interest. The code has been written with downloading data for multiple months and combining them into one CSV file in mind.
base_url = "https://s3.amazonaws.com/tripdata/"
df_li = []

# Limit the scope of the study to June 2022.
year = '2022'
# Loop through the period of interest by month. For a whole year of data, specify np.arange(1,13).
for i in np.arange(6,7):
    if i not in [6, 7]:
        csv_name = f'{year}{i:02d}-citibike-tripdata.csv'
    else:
        # The file names for June and July of 2022 are different from those for the other months due to typo.
        csv_name = f'{year}{i:02d}-citbike-tripdata.csv'
    
    # The Citi Bike system data are stored as zip files.
    zip_name = csv_name + '.zip'
    zip_url = base_url + zip_name
    
    # Request the zip file and extract its content.
    zip_response = requests.get(zip_url)
    with open(zip_name, 'wb') as f:
        f.write(zip_response.content)
    with zipfile.ZipFile(zip_name, 'r') as zip:
        zip.extractall(year)
    
    # Import the extract CSV file and create a DataFrame for it.
    df = pd.read_csv(os.path.join(year, csv_name), dtype={'start_station_id': str, 'end_station_id': str})
    # 'ride_id' can be dropped immediately as it is used to identify trips, as do the DataFrame indices.
    df.drop(columns=['ride_id'], inplace=True)
    
    df_li.append(df)
    os.remove(zip_name)
    os.remove(os.path.join(year, csv_name))
    
# Concatenate all monthly data.
df = pd.concat(df_li, axis=0, ignore_index=True)

# Edited: import the additional JC (Jersey City) data for June 2022.
# Upon close investigation of 202206-citibike-tripdata.csv, it is noted that a few trips that ended in JC are included in the
# dataset. Because Citi Bike extends to Jersey City and Hoboken in New Jersey, the JC data, which share the same base URL with 
# the NYC data, are included for the time period investigated.
csv_name = f'JC-{year}06-citibike-tripdata.csv'
zip_name = csv_name + '.zip'
zip_response = requests.get(base_url + zip_name)

with open(zip_name, 'wb') as f:
    f.write(zip_response.content)
with zipfile.ZipFile(zip_name, 'r') as zip:
    zip.extractall(year)
    
df_jc = pd.read_csv(os.path.join(year, csv_name), dtype={'start_station_id': str, 'end_station_id': str})
df_jc.drop(columns=['ride_id'], inplace=True)

os.remove(zip_name)
os.remove(os.path.join(year, csv_name))

df = pd.concat([df, df_jc], axis=0, ignore_index=True)
df.to_csv(os.path.join(year, f'{year}06-citibike-tripdata.csv'), index=False)

# Transform

In [4]:
# Import the ridership CSV file.
df = pd.read_csv(os.path.join('2022', '202206-citibike-tripdata.csv'), dtype={'start_station_id': str, 'end_station_id': str})

## Trip Duration Outliers

In [5]:
df_1 = df.copy()

# Convert start time and end time to datetime objects to calculate trip duration.
df_1['started_at'] = pd.to_datetime(df_1['started_at'])
df_1['ended_at'] = pd.to_datetime(df_1['ended_at'])
df_1['trip_duration'] = (df_1['ended_at'] - df_1['started_at']).dt.total_seconds().astype('int64')

# The dataset is still relatively new, so it still needs to be processed to remove trips below 60 seconds in length as per
# https://citibikenyc.com/system-data. There are trips lasting days in the dataset, which are clearly outliers, so an upper
# limit of one day is set as well.
df_1 = df_1[(df_1['trip_duration']>=60) & (df_1['trip_duration']<=(60*60*24))]

# The type of bikes used for each trip is outside the scope of the current study, so it will be dropped.
df_1.drop(columns=['rideable_type'], inplace=True)

df_1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3441935 entries, 0 to 3536181
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   started_at          3441935 non-null  datetime64[ns]
 1   ended_at            3441935 non-null  datetime64[ns]
 2   start_station_name  3441935 non-null  object        
 3   start_station_id    3441935 non-null  object        
 4   end_station_name    3439610 non-null  object        
 5   end_station_id      3439610 non-null  object        
 6   start_lat           3441935 non-null  float64       
 7   start_lng           3441935 non-null  float64       
 8   end_lat             3441654 non-null  float64       
 9   end_lng             3441654 non-null  float64       
 10  member_casual       3441935 non-null  object        
 11  trip_duration       3441935 non-null  int64         
dtypes: datetime64[ns](2), float64(4), int64(1), object(5)
memory usage: 34

In [6]:
# Drop rows without end station info as the study focuses on trips that start and end at bike stations.
df_1.dropna(inplace=True)

In [7]:
df_start = df_1.iloc[:, [0,3,2,6,7,10,11]]
df_start = df_start.assign(start_end='start')
df_start.rename(columns={'started_at':'time_stamp', 
                         'start_station_id':'station_id',
                         'start_station_name':'station_name',
                         'start_lat':'latitude',
                         'start_lng':'longitude'}, inplace=True)

In [8]:
df_end = df_1.iloc[:, [1,5,4,8,9,10,11]]
df_end = df_end.assign(start_end='end')
df_end.rename(columns={'ended_at':'time_stamp', 
                       'end_station_id':'station_id',
                       'end_station_name':'station_name',
                       'end_lat':'latitude',
                       'end_lng':'longitude'}, inplace=True)

In [9]:
df_1 = pd.concat([df_start, df_end], axis=0, ignore_index=True)

In [10]:
df_1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6879220 entries, 0 to 6879219
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   time_stamp     6879220 non-null  datetime64[ns]
 1   station_id     6879220 non-null  object        
 2   station_name   6879220 non-null  object        
 3   latitude       6879220 non-null  float64       
 4   longitude      6879220 non-null  float64       
 5   member_casual  6879220 non-null  object        
 6   trip_duration  6879220 non-null  int64         
 7   start_end      6879220 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 419.9+ MB


## Station Names and Ids
To ensure accurate groupby operations downstream, each station name should be related to its id one-to-one. However, the number of unique names is greater than the number of unique ids, indicating multiple station names referring to the same id and possibly vice versa.

In [11]:
# Check the unique station names and ids.
print(f"Number of unique station names: {df_1['station_name'].nunique()}")
print(f"Number of unique station ids: {df_1['station_id'].nunique()}")

Number of unique station names: 1650
Number of unique station ids: 1642


In [12]:
# Determine the start station ids that multiple station names refer to.
# If the station name is related to its id one-to-one, then df_1[['start_station_name', 'start_station_id']].drop_duplicates()
# should have the same index sequence as df_1['start_station_id'].drop_duplicates(). However, that is not the case: the 
# additional station names need to be addressed.
diff_id = df_1[['station_name', 'station_id']].drop_duplicates().index.difference(
    df_1['station_id'].drop_duplicates().index)
diff = df_1.loc[diff_id]['station_id'].sort_values()

# Print the start station ids with their conflicting names.
# repr() is used to prevent escape sequence interpretation.
for i in diff:
    print(i + ': ' + 
          ', '.join(repr(name) for name in df_1[df_1['station_id'] == i]['station_name'].unique()))

4488.09: 'Boerum Pl\\t& Pacific St', 'Boerum Pl\t& Pacific St'
4781.05: 'Nassau St\\t& Duffield St', 'Nassau St\t& Duffield St'
5323.06: 'Sharon St & Olive St', 'Sharon St & Olive St_new'
5329.08: 'Murray St\\t& West St', 'Murray St\t& West St'
5382.07: 'Forsyth St\t& Grand St', 'Forsyth St\\t& Grand St'
5883.06: 'Van Dam St & Greenpoint Ave', 'Van Dam St & Review Ave'
6300.04: 'Skillman Ave & 43 Ave', 'Skillman Ave & 32 Pl'
6535.04: 'W 34 St &\\tHudson Blvd E', 'W 34 St &\tHudson Blvd E'
6560.14: 'W 40 St & 7 Ave', 'W 40 St & 8 Ave'
6708.04: 'Broadway\\t& W 48 St', 'Broadway\t& W 48 St'


In [13]:
# For 6 of them, it is a simple matter of replacing '\\t' with '\t' (and removing '\t' altogether in station names), and the
# rest can be adjusted using the station information as reference.
df_1['station_name'] = df_1['station_name'].str.replace(r'\\t', r'\t', regex=True)
df_1['station_name'] = df_1['station_name'].str.replace(r'\t', ' ', regex=True)

In [14]:
# Import the station information and use it as a reference to resolve the remaining conflicting station names.
station_info = pd.read_json(os.path.join('data', 'station_info.json')).loc[:, ['short_name', 'name', 'lat', 'lon']]
station_info.rename(columns={'short_name':'station_id'}, inplace=True)

# Note that the list can be generated by the same code used to determine the station ids that multiple station names refer to.
for i in ['5323.06', '5883.06', '6300.04', '6560.14']:
    correct_name = station_info[station_info['station_id'] == i].name.str.cat()
    print(i + ': ' + correct_name)
    df_1.loc[df_1['station_id'] == i, 'station_name'] = correct_name

5323.06: Sharon St & Olive St
5883.06: Van Dam St & Greenpoint Ave
6300.04: Skillman Ave & 43 Ave
6560.14: W 40 St & 7 Ave


In [15]:
# Determine the station names that multiple station ids refer to.
diff_id = df_1[['station_name', 'station_id']].drop_duplicates().index.difference(
    df_1['station_name'].drop_duplicates().index)
diff = df_1.loc[diff_id]['station_name'].sort_values()

# Print the start station names and their conflicting ids.
for name in diff:
    print(name + ': ' + ', '.join(i for i in df_1[df_1['station_name'] == name]['station_id'].unique()))

30 Ave & 12 St: 7034.08, 7034.09
Sharon St & Olive St: 5323.05, 5323.06


In [16]:
# Once again, use the station information to resolve the conflicting station ids.
for name in diff:
    correct_id = station_info[station_info['name'] == name]['station_id'].str.cat()
    print(name + ': ' + correct_id)
    df_1.loc[df_1['station_name'] == name, 'station_id'] = correct_id

30 Ave & 12 St: 7034.09
Sharon St & Olive St: 5323.06


In [17]:
# Check the unique station names and ids again.
print(f"Number of unique station names: {df_1['station_name'].nunique()}")
print(f"Number of unique station ids: {df_1['station_id'].nunique()}")

Number of unique station names: 1640
Number of unique station ids: 1640


## Station Coordinates

In [18]:
df_2 = df_1.copy()

# Inspect the starting latitude and longitude for trips started from 5382.07.
df_2[df_2['station_id'] == '5382.07'][['latitude', 'longitude']].drop_duplicates()

,latitude,longitude
312415,40.717798,-73.993161
328252,40.717798,-73.993161
360129,40.717444,-73.993426
376208,40.717684,-73.993301
397618,40.717780,-73.993254
...,...,...
3175873,40.717781,-73.993242
3195730,40.717516,-73.993466
3302830,40.717567,-73.993388
3333817,40.717710,-73.993271


Multiple pairs of latitude and longitude are tied to the same station name, but they are in close proximity, indicating that they refer to the coordinates of trips made to/from the same station. In order to map the incoming and outgoing traffic from bike stations, it is easier to work with a single set of coordinates for each station. To that end, merge the `df_2` and `station_info` DataFrames. The missing station coordinates can be imputed by averaging the trip coordinates associated with each station.

As proof of concept, compare the latitude and longitude of `5382.07` from the station information with the averaged trip coordinates tied to the station: they are similar down to the fifth decimal place, which implies accuracy of about 1.1 m in physical space.

In [19]:
station_info.loc[station_info['station_id'] == '5382.07'][['lat', 'lon']]

,lat,lon
1123,40.717798,-73.993161


In [20]:
df_2[df_2['station_id'] == '5382.07'][['latitude', 'longitude']].mean()

latitude     40.717799
longitude   -73.993163
dtype: float64

In [21]:
# Left outer merge station_info into df_2.
stations = station_info.copy()
stations = stations.loc[:, ['station_id', 'lat', 'lon']]
stations.rename(columns={'lat':'station_lat', 'lon':'station_lon'}, inplace=True)
df_2 = pd.merge(df_2, stations, on='station_id', how='left')

In [22]:
df_2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6879220 entries, 0 to 6879219
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   time_stamp     6879220 non-null  datetime64[ns]
 1   station_id     6879220 non-null  object        
 2   station_name   6879220 non-null  object        
 3   latitude       6879220 non-null  float64       
 4   longitude      6879220 non-null  float64       
 5   member_casual  6879220 non-null  object        
 6   trip_duration  6879220 non-null  int64         
 7   start_end      6879220 non-null  object        
 8   station_lat    6705127 non-null  float64       
 9   station_lon    6705127 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 577.3+ MB


In [23]:
# Fill in missing start station coordinates with the average trip start coordinates.
df_2['station_lat'].fillna(df_2[df_2['station_lat'].isna()].groupby('station_id')['latitude'].transform('mean'), inplace=True)
df_2['station_lon'].fillna(df_2[df_2['station_lon'].isna()].groupby('station_id')['longitude'].transform('mean'), inplace=True)

In [24]:
df_2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6879220 entries, 0 to 6879219
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   time_stamp     6879220 non-null  datetime64[ns]
 1   station_id     6879220 non-null  object        
 2   station_name   6879220 non-null  object        
 3   latitude       6879220 non-null  float64       
 4   longitude      6879220 non-null  float64       
 5   member_casual  6879220 non-null  object        
 6   trip_duration  6879220 non-null  int64         
 7   start_end      6879220 non-null  object        
 8   station_lat    6879220 non-null  float64       
 9   station_lon    6879220 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 577.3+ MB


In [25]:
# Drop the trip coordinates as the unique station coordinates have been determined.
df_2.drop(columns=['latitude', 'longitude'], inplace=True)

# Reorganize the columns.
df_2 = df_2.iloc[:, [0,1,2,6,7,3,4,5]]

In [26]:
df_2.head()

,time_stamp,station_id,station_name,station_lat,station_lon,member_casual,trip_duration,start_end
0,2022-06-08 18:55:00,7884.04,E 149 St & Park Ave,40.818154,-73.925294,member,193,start
1,2022-06-18 16:34:50,7599.02,E 115 St & Madison Ave,40.798944,-73.944846,member,779,start
2,2022-06-11 17:57:00,7599.02,E 115 St & Madison Ave,40.798944,-73.944846,member,633,start
3,2022-06-23 17:32:04,5569.06,W Broadway & Spring St,40.724947,-74.001659,member,392,start
4,2022-06-30 16:17:43,5779.10,E 14 St & 1 Ave,40.731393,-73.982867,member,615,start


# Load

In [27]:
df_2.to_csv(os.path.join('data', '202206-citibike-tripdata-cleaned.csv'), index=False)